In [29]:

from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem.snowball import SnowballStemmer

import pandas as pd
import nltk

nltk.download('stopwords')

stemmer = SnowballStemmer("english")


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Enes\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [30]:

# importing the dataset
DATASET_COLUMNS  = ["sentiment", "ids", "date", "flag", "user", "tweet"]
DATASET_ENCODING = "ISO-8859-1"

df = pd.read_csv('./training.1600000.processed.noemoticon.csv',
                      delimiter=',', encoding=DATASET_ENCODING, names=DATASET_COLUMNS)

# df = pd.read_csv('./IMDB Dataset.csv', delimiter=',',
#                       encoding=DATASET_ENCODING)

# df = pd.read_csv('./Corona_NLP_train.csv', delimiter=',', encoding=DATASET_ENCODING)

# removing the unnecessary columns.
df = df[['tweet', 'sentiment']]
# df = dataset[['review','sentiment']]
# df = dataset[['OriginalTweet','Sentiment']]

dataset_dir = 'sentiment140'
# dataset_dir = 'imdb'
# dataset_dir = 'coronaNLP'

# n_gram = (1, 1)
n_gram = (1, 2)
# n_gram = (2, 2)

vector_dir = './vectors/'+dataset_dir

In [31]:

df.drop_duplicates()

df.head()


,tweet,sentiment
0,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",0
1,is upset that he can't update his Facebook by ...,0
2,@Kenichan I dived many times for the ball. Man...,0
3,my whole body feels itchy and like its on fire,0
4,"@nationwideclass no, it's not behaving at all....",0


In [32]:
# Preprocessing
from nltk.corpus import stopwords
import re
import string


def remove_URL(text):
    url = re.compile(r"https?://\S+|www\.\S+")
    return url.sub(r"", text)


def remove_punct(text):
    translator = str.maketrans("", "", string.punctuation)
    return text.translate(translator)


def remove_mention(text):
    return re.sub("@[A-Za-z0-9]+", "", text)


def stem_tweets(tweet):
    tokens = tweet.split()
    stemmed_tokens = [stemmer.stem(token) for token in tokens]
    return ' '.join(stemmed_tokens)


def lemmatize_tweets(tweet):
    tokens = tweet.split()
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]
    return ' '.join(lemmatized_tokens)

# remove stopwords


stop = set(stopwords.words("english"))


def remove_stopwords(text):
    stop = set(stopwords.words("english"))

    filtered_words = [word.lower()
                      for word in text.split() if word.lower() not in stop]
    return " ".join(filtered_words)


def preprocess_tweets(tweet):
    tweet = remove_mention(tweet)
    tweet = remove_URL(tweet)
    tweet = remove_punct(tweet)
    tweet = stem_tweets(tweet)
    # tweet = lemmatize_tweets(tweet)
    tweet = remove_stopwords(tweet)
    return tweet


In [33]:
df.iloc[:, 0] = df.iloc[:, 0].apply(preprocess_tweets)

X = df.iloc[:, 0]

y = df.iloc[:, 1]

df.head()


,tweet,sentiment
0,awww bummer shoulda got david carr third day,0
1,upset cant updat facebook text might cri resul...,0
2,dive mani time ball manag save 50 rest go bound,0
3,whole bodi feel itchi like fire,0
4,behav im mad whi becaus cant see,0


In [34]:

from nltk.tokenize import RegexpTokenizer

token = RegexpTokenizer(r'[a-zA-Z0-9]+')

tfidf = TfidfVectorizer(max_features=50000, ngram_range=n_gram, tokenizer=token.tokenize)
tfidf

TfidfVectorizer(max_features=50000, ngram_range=(1, 2),
                tokenizer=<bound method RegexpTokenizer.tokenize of RegexpTokenizer(pattern='[a-zA-Z0-9]+', gaps=False, discard_empty=True, flags=<RegexFlag.UNICODE|DOTALL|MULTILINE: 56>)>)

In [35]:
tfidf.fit(X)

C:\Users\Enes\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\feature_extraction\text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  "The parameter 'token_pattern' will not be used"


TfidfVectorizer(max_features=50000, ngram_range=(1, 2),
                tokenizer=<bound method RegexpTokenizer.tokenize of RegexpTokenizer(pattern='[a-zA-Z0-9]+', gaps=False, discard_empty=True, flags=<RegexFlag.UNICODE|DOTALL|MULTILINE: 56>)>)

In [36]:
tfidf

TfidfVectorizer(max_features=50000, ngram_range=(1, 2),
                tokenizer=<bound method RegexpTokenizer.tokenize of RegexpTokenizer(pattern='[a-zA-Z0-9]+', gaps=False, discard_empty=True, flags=<RegexFlag.UNICODE|DOTALL|MULTILINE: 56>)>)

In [37]:
import joblib

joblib.dump(tfidf, f'./vectors/vectorizer_{dataset_dir}_{n_gram}.pkl')


['./vectors/vectorizer_sentiment140_(1, 2).pkl']